In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import OLSInfluence
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

<font size="8" color="Black">Part 1 and 2</font>

In [3]:
# Extract data of housing price average f| 5 most populated cities in US
#pd.set_option('display.max_rows', 500)
prices = pd.read_csv('Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv').head(6).tail(-1)
prices = pd.melt(prices, id_vars=['RegionName'], value_vars=prices.columns[5:])
prices['Date'] = pd.to_datetime(prices['variable'])
prices['Date'] = prices['Date'].astype('str')
prices = prices.drop('variable', axis=1).rename(columns={'value': 'Average Price'})
prices['Date'] = pd.DatetimeIndex(prices['Date'])
prices['Year'] = pd.DatetimeIndex(prices['Date']).year
prices.drop(columns='Date', axis=1)
display(prices)
prices = prices.groupby(['Year', 'RegionName']).aggregate(np.mean)
prices = prices.reset_index()
# Plot the avergae house price per month for each city
fig, ax = plt.subplots()
prices = prices.rename(columns={'RegionName': 'City'})
prices.groupby('City')['Average Price'].plot(x='Year', y='Average Price', figsize=(30, 15), ax=ax)
plt.legend(prop={'size': 15})
plt.rcParams.update({'font.size': 20})
plt.show()
prices

FileNotFoundError: [Errno 2] No such file or directory: 'Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv'

This is our code to gather data on the median house price for each of our 5 cities since 2000. All it is is a bunch
of data tidying to organize our table into a readable format. We have also included a graph that shows how each city's
median house price has changed based on the table. 

In [ ]:
# This function drops rows that combine multiple cities (which makes no sense) after melting
def drop_invalid(df):
    cities = [] 
    for index, row in df.iterrows():
        if ('New York' in row['variable'] and row['City'] == 'New York, NY'):
            cities.append('New York, NY')
        elif ('Los Angeles' in row['variable'] and row['City'] == 'Los Angeles, CA'):
            cities.append('Los Angeles, CA')
        elif ('Chicago' in row['variable'] and row['City'] == 'Chicago, IL'):
            cities.append('Chicago, IL')
        elif ('Houston' in row['variable'] and row['City'] == 'Houston, TX'):
            cities.append('Houston, TX')
        elif ('Dallas' in row['variable'] and row['City'] == 'Dallas, TX'):
            cities.append('Dallas, TX')
        else:
            df = df.drop(index)
     
    df['City'] = cities      
    return df.drop('variable',axis=1)


# Read the data 
df = pd.read_csv('demographics.csv')
df['Year'] = pd.DatetimeIndex(df['date']).year
# Get recent data 
df = df[(df['Year'] > 2011)] 
df = df.drop('date', axis=1)

# DATA TIDYING. VERY MESSY, LOTS OF MELTING, BUT NECESSARY
df2 = df.head(10)
df = df.tail(-10)
df = df.groupby('Year').aggregate(np.mean)
df = df.fillna(df2.set_index('Year')).reset_index()
            
df = pd.melt(df, id_vars=['Year',
       'Los Angeles Count_Person_EducationalAttainmentBachelorsDegreeOrHigher',
       'Los Angeles Count_Person',
       'Chicago Count_Person_EducationalAttainmentBachelorsDegreeOrHigher',
       'Chicago Count_Person',
       'New York City Count_Person_EducationalAttainmentBachelorsDegreeOrHigher',
       'New York City Count_Person',
       'Dallas Count_Person_EducationalAttainmentBachelorsDegreeOrHigher',
       'Dallas Count_Person',
       'Houston Count_Person_EducationalAttainmentBachelorsDegreeOrHigher',
       'Houston Count_Person', 'Los Angeles UnemploymentRate_Person',
       'Chicago UnemploymentRate_Person',
       'New York City UnemploymentRate_Person',
       'Dallas UnemploymentRate_Person', 'Houston UnemploymentRate_Person',
       'Los Angeles Median_Income_Household',
       'Chicago Median_Income_Household',
       'New York City Median_Income_Household',
       'Dallas Median_Income_Household', 'Houston Median_Income_Household'], value_name='Population under poverty line', 
             value_vars=['Los Angeles Count_Person_BelowPovertyLevelInThePast12Months', 'Chicago Count_Person_BelowPovertyLevelInThePast12Months',
             'New York City Count_Person_BelowPovertyLevelInThePast12Months',
            'Houston Count_Person_BelowPovertyLevelInThePast12Months',
            'Dallas Count_Person_BelowPovertyLevelInThePast12Months'])  

cities = [] 
for index, row in df.iterrows():
    if ('New York City' in row['variable']):
        cities.append('New York, NY')
    elif ('Los Angeles' in row['variable']):
        cities.append('Los Angeles, CA')
    elif ('Chicago' in row['variable']):
        cities.append('Chicago, IL')
    elif ('Houston' in row['variable']):
        cities.append('Houston, TX')
    else:
        cities.append('Dallas, TX')
     
df['City'] = cities      
df = df.drop('variable',axis=1)

df = pd.melt(df, id_vars=['Year', 'Population under poverty line', 'Los Angeles Count_Person', 'City',
                        'Chicago Count_Person',
                        'New York City Count_Person',
                        'Dallas Count_Person',
                        'Houston Count_Person', 'Los Angeles UnemploymentRate_Person',
                        'Chicago UnemploymentRate_Person',
                        'New York City UnemploymentRate_Person',
                        'Dallas UnemploymentRate_Person', 'Houston UnemploymentRate_Person',
                        'Los Angeles Median_Income_Household',
                        'Chicago Median_Income_Household',
                        'New York City Median_Income_Household',
                        'Dallas Median_Income_Household', 'Houston Median_Income_Household'], value_name='Population with Bachelors or Higher',       
             value_vars=['Los Angeles Count_Person_EducationalAttainmentBachelorsDegreeOrHigher',         
                        'Chicago Count_Person_EducationalAttainmentBachelorsDegreeOrHigher',
                        'New York City Count_Person_EducationalAttainmentBachelorsDegreeOrHigher',
                        'Dallas Count_Person_EducationalAttainmentBachelorsDegreeOrHigher',
                        'Houston Count_Person_EducationalAttainmentBachelorsDegreeOrHigher'])

df = drop_invalid(df)

df = pd.melt(df, id_vars=['Year', 'Population under poverty line', 'Population with Bachelors or Higher', 'City',
                            'Los Angeles UnemploymentRate_Person',
                            'Chicago UnemploymentRate_Person',
                            'New York City UnemploymentRate_Person',
                            'Dallas UnemploymentRate_Person', 'Houston UnemploymentRate_Person',
                            'Los Angeles Median_Income_Household',
                            'Chicago Median_Income_Household',
                            'New York City Median_Income_Household',
                            'Dallas Median_Income_Household', 'Houston Median_Income_Household'], 
             value_name='Total Population', value_vars=['Los Angeles Count_Person',
                                                        'Chicago Count_Person',
                                                      'New York City Count_Person',
                                                        'Dallas Count_Person',
                                                        'Houston Count_Person'])
df = drop_invalid(df)
df = pd.melt(df, id_vars=['Year', 'Population under poverty line', 'Population with Bachelors or Higher', 'Total Population', 'City',
                        'Los Angeles Median_Income_Household',
                        'Chicago Median_Income_Household',
                        'New York City Median_Income_Household',
                        'Dallas Median_Income_Household', 'Houston Median_Income_Household'], 
             value_name='Unemployment Rate', value_vars=['Los Angeles UnemploymentRate_Person',
                                                        'Chicago UnemploymentRate_Person',
                                                        'New York City UnemploymentRate_Person',
                                                        'Dallas UnemploymentRate_Person', 'Houston UnemploymentRate_Person'])
df = drop_invalid(df)
df = pd.melt(df, id_vars=['Year', 'City', 'Population under poverty line', 'Population with Bachelors or Higher', 'Total Population', 'Unemployment Rate'], 
             value_name='Median Household Income', value_vars=['Los Angeles Median_Income_Household',
                                   'Chicago Median_Income_Household',
                                   'New York City Median_Income_Household',
                                   'Dallas Median_Income_Household', 'Houston Median_Income_Household'])
df = drop_invalid(df)

for index, row in df.iterrows():
    if (pd.isna(row['Population with Bachelors or Higher']) or pd.isna(row['Total Population']) or pd.isna(row['Unemployment Rate']) or pd.isna(row['Median Household Income'])):
        df = df.drop(index)
df = df.reset_index().drop('index', axis=1)
df['Percent of Population with Bachelors or Higher'] = 100 * df['Population with Bachelors or Higher'] / df['Total Population']
df['Percent of Population Under Poverty Line'] = 100 * df['Population under poverty line'] / df['Total Population']
df = df.drop('Population with Bachelors or Higher', axis=1).drop('Population under poverty line', axis=1)
df

We gathered this data off using a census tool that allows us to combine data into tables, which is how we got the csv.
Although we initially had all the data in the table, it was unusable. In order to make it usable and readable, we had to
melt it a good amount as you can see above. Although it was messy, we were able to tidy the data into one nice table that shows
demographics data on the cities from 2012-2020.

In [ ]:
final_df = df.merge(prices, on=["City", "Year"])
final_df

Now, all we got to do is merge the two tables into one table with all our data for each year. This is what we will use 
for the future parts.

<font size="8" color="Black">Part 3</font>

After finishing the table, lets analyze some information in the table that might impact the price of the house in 
different cities. First, we want to do so visually by plotting out each factor. 

In [ ]:
# Draw a line plot of Average Price From 2012-2020
city_set = set(final_df['City'])

plt.figure(figsize=(30,15)) 
for city in city_set:
     selected_data = final_df.loc[final_df['City'] == city]
     plt.plot(selected_data['Year'], selected_data['Average Price'], label=city)
    
plt.title("Average Price From 2012-2020")
plt.xlabel("Year")
plt.ylabel("Average Price")
plt.legend()
plt.show()

In [ ]:
# Draw a line plot of Total Population From 2012-2020
city_set = set(final_df['City'])

plt.figure(figsize=(30,15)) 
for city in city_set:
     selected_data = final_df.loc[final_df['City'] == city]
     plt.plot(selected_data['Year'], selected_data['Total Population'], label=city)
    
plt.title("Total Population From 2012-2020")
plt.xlabel("Year")
plt.ylabel("Total Population (million people)")
plt.legend()
plt.show()

In [ ]:
# Draw a line plot of Unemployment Rate From 2012-2020
city_set = set(final_df['City'])

plt.figure(figsize=(30,15)) 
for city in city_set:
     selected_data = final_df.loc[final_df['City'] == city]
     plt.plot(selected_data['Year'], selected_data['Unemployment Rate'], label=city)
    
plt.title("Unemployment Rate From 2012-2020")
plt.xlabel("Year")
plt.ylabel("Unemployment Rate")
plt.legend()
plt.show()

In [ ]:
# Draw a line plot of Median Household Income From 2012-2020
city_set = set(final_df['City'])

plt.figure(figsize=(30,15)) 
for city in city_set:
     selected_data = final_df.loc[final_df['City'] == city]
     plt.plot(selected_data['Year'], selected_data['Median Household Income'], label=city)
    
plt.title("Median Household Income From 2012-2020")
plt.xlabel("Year")
plt.ylabel("Median Household Income")
plt.legend()
plt.show()

In [ ]:
# Draw a line plot of Percent of Population with Bachelors or Higher From 2012-2020
city_set = set(final_df['City'])

plt.figure(figsize=(30,15)) 
for city in city_set:
     selected_data = final_df.loc[final_df['City'] == city]
     plt.plot(selected_data['Year'], selected_data['Percent of Population with Bachelors or Higher'], label=city)
    
plt.title("Percent of Population with Bachelors or Higher From 2012-2020")
plt.xlabel("Year")
plt.ylabel("Percent of Population with Bachelors or Higher")
plt.legend()
plt.show()

In [ ]:
# Draw a line plot of Percent of Population Under Poverty Line From 2012-2020
city_set = set(final_df['City'])

plt.figure(figsize=(30,15)) 
for city in city_set:
     selected_data = final_df.loc[final_df['City'] == city]
     plt.plot(selected_data['Year'], selected_data['Percent of Population Under Poverty Line'], label=city)
    
plt.title("Percent of Population Under Poverty Line From 2012-2020")
plt.xlabel("Year")
plt.ylabel("Percent of Population Under Poverty Line")
plt.legend()
plt.show()

From the graph above, we can know that the average price has been increasing from 2012-2020. Total Population 
doesn't change too much. Unemployment rate decerased and increased. Median household Income and Percent of Population with Bachelors increases consistently; Percent of Population Under Poverty Line decreases consistently from 2012-2020.

Since the average house price has a trend that is increasing constantly, we want to find out that what might be the reasons leads to the increaing price. So we are going to draw a correlation matrix to find out the relationships between those factors and average house price.

In [ ]:
plt.figure(figsize=(30,15))
correlation_matrix = final_df.corr()
sns.heatmap(correlation_matrix, annot=True)
plt.show()

The correlation matrix shows us that there are 4 factors that might be able to impact the housing price. Those are:Year, Totoal Population, Median Household Income, Percent of Population with Bachelors or Higher, Percent of population Under Proverty. In these factor, Median Household Income has a strong relationship with Average price.

<font size="8" color="Black">Part 4</font>

We are going to build an initial regression model to predict the house price between differnet cities. 

In [ ]:
# Label encoding for the categorical variable 'City'
le = LabelEncoder()
final_df['City'] = le.fit_transform(final_df['City'])

# Defining predictors and target variable
X = final_df[['City', 'total_population', 'median_household_income', 'percent_bachelors_or_higher', 
        'unemployment_rate', 'percent_under_poverty_line', 'Year']]
y = final_df['average_price']

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Defining the model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Fitting the model
model.fit(X_train, y_train)

# Predicting the test set results
y_pred = model.predict(X_test)

# Evaluating the model
mse = mean_squared_error(y_test, y_pred)
print("MSE: ", mse)
print("RMSE: ", mse**(0.5))

The Mean Squared Error (MSE) and Root Mean Squared Error (RMSE) are both measures of model accuracy, with lower values indicating better performance.

The model's MSE is 2861185276.97, and the RMSE is 53490.05. The RMSE can be interpreted in the same units as the dependent variable, which in this case is the average house price. This means that, on average, your model's predictions are approximately $53,490 off from the actual house price.

In [ ]:
average_predicted_price = np.mean(y_pred)
print("The average predicted house price is: ", average_predicted_price)

In [ ]:
# Assuming you have a new data point for the new city
new_city_data = {'City': [4], 'total_population': [8433867], 'median_household_income': [63998], 
                 'percent_bachelors_or_higher': [0.27351827], 'unemployment_rate': [0.441667], 
                 'percent_under_poverty_line': [0.17116053], 'Year': [2023]}

new_df = pd.DataFrame(new_city_data)

# Convert all 'City' values in the original dataframe to strings and perform label encoding
final_df['City'] = final_df['City'].astype(str)
le = LabelEncoder()
final_df['City'] = le.fit_transform(final_df['City'])

# Separate predictors and target variable for the original data and train the model
X = final_df[['City', 'total_population', 'median_household_income', 'percent_bachelors_or_higher', 
        'unemployment_rate', 'percent_under_poverty_line', 'Year']]
y = final_df['average_price']

# Fit the model
model.fit(X, y)

# Convert 'City' in new data to string, encode it and make a prediction
new_df['City'] = new_df['City'].astype(str)
new_city_encoded = le.transform(new_df['City'])
new_df['City'] = new_city_encoded

# Now you can predict the house price for the new city
new_city_prediction = model.predict(new_df)

# Print out the prediction
print("Predicted average house price for the new city: ", new_city_prediction[0])

In [ ]:
X

Let's try some tactics that might be able to improve the accruary of the prediction model

In [ ]:
final_df['income_per_population'] = final_df['median_household_income'] / final_df['total_population']

In [ ]:
# Don't forget to add the new feature to the new city data as well
new_df['income_per_population'] = new_df['median_household_income'] / new_df['total_population']

# Redefine your feature set to include the new feature
X = final_df[['City', 'total_population', 'median_household_income', 'percent_bachelors_or_higher', 'unemployment_rate', 'percent_under_poverty_line', 'Year', 'income_per_population']]

# Fit the model to the data
model.fit(X, y)

# Now you can predict the house price for the new city using the updated model
new_city_prediction = model.predict(new_df)

# Print out the prediction
print("Predicted average house price for the new city: ", new_city_prediction[0])

<font size="8" color="Black">Part 5</font>

<font size="8" color="Black">Part 6</font>